In [1]:
! pip install rank_bm25
! pip install transformers
#! pip install torch
from transformers import BertTokenizer, BertModel

import urllib.request as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np

import nltk
import torch
from sklearn.metrics import ndcg_score
from collections import defaultdict

nltk.download('stopwords')
nltk.download('punkt')

nb_docs=150

def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename,encoding='utf-8') as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename,encoding='utf-8') as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename, encoding='utf-8') as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rosel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rosel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## First step we run the code given as an example

In [2]:

def text_to_token_list(text):
    stopword = stopwords.words('english')
    word_tokens = word_tokenize(text.lower())
    word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word) > 2]
    return word_tokens_without_stops

def run_bm25_only(start_doc, end_doc):
    dic_doc, dic_req, dic_req_doc = loadNFCorpus()

    docs_to_keep = []
    reqs_to_keep = []
    dic_req_doc_to_keep = defaultdict(dict)

    ndcg_top = 5
    i = start_doc

    for req_id in dic_req_doc:
        if i > (end_doc - start_doc):
            break
        for doc_id in dic_req_doc[req_id]:
            dic_req_doc_to_keep[req_id][doc_id] = dic_req_doc[req_id][doc_id]
            docs_to_keep.append(doc_id)
            i += 1
        reqs_to_keep.append(req_id)

    docs_to_keep = list(set(docs_to_keep))

    all_vocab_doc = {}
    for k in docs_to_keep:
        doc_token_list = text_to_token_list(dic_doc[k])
        for word in doc_token_list:
            if word not in all_vocab_doc:
                all_vocab_doc[word] = word

    all_vocab_list_doc = list(all_vocab_doc)

    all_vocab_req = {}
    for k in reqs_to_keep:
        req_token_list = text_to_token_list(dic_req[k])
        for word in req_token_list:
            if word not in all_vocab_req:
                all_vocab_req[word] = word

    all_vocab_list_req = list(all_vocab_req)

    from rank_bm25 import BM25Okapi

    corpus_doc_token_list = []
    corpus_req_token_list = {}
    corpus_doc_name = []
    corpus_dico_doc_name = {}
    i = 0

    for k in docs_to_keep:
        doc_token_list = text_to_token_list(dic_doc[k])
        corpus_doc_token_list.append(doc_token_list)
        corpus_doc_name.append(k)
        corpus_dico_doc_name[k] = i
        i += 1

    corpus_req_name = []
    corpus_dico_req_name = {}
    i = 0

    for k in reqs_to_keep:
        req_token_list = text_to_token_list(dic_req[k])
        corpus_req_token_list[k] = req_token_list
        corpus_req_name.append(k)
        corpus_dico_req_name[k] = i
        i += 1

    bm25 = BM25Okapi(corpus_doc_token_list)

    ndcg_cumul = 0
    corpus_req_vec = {}
    ndcg_bm25_cumul = 0
    nb_req = 0

    
    for req in corpus_req_token_list:
        req_token_list = corpus_req_token_list[req]
        doc_scores = bm25.get_scores(req_token_list)
        true_docs = np.zeros(len(corpus_doc_token_list))

        for doc_id in corpus_dico_doc_name:
            if req in dic_req_doc_to_keep and doc_id in dic_req_doc_to_keep[req]:
                pos_doc_id = corpus_dico_doc_name[doc_id]
                true_docs[pos_doc_id] = dic_req_doc_to_keep[req][doc_id]

        ndcg_bm25_cumul += ndcg_score([true_docs], [doc_scores], k=ndcg_top)
        nb_req += 1

    ndcg_bm25_cumul /= nb_req
    print("ndcg bm25=", ndcg_bm25_cumul)
    return ndcg_bm25_cumul


run_bm25_only(0, nb_docs)


ndcg bm25= 0.8135524489389909


0.8135524489389909

## Définition des approches BERT à tester 

In [3]:
tokenizer_base= BertTokenizer.from_pretrained('bert-base-uncased')
model_base= BertModel.from_pretrained('bert-base-uncased')


C:\Users\rosel\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rosel\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
bio_bert_model='pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb'

tokenizer_bio= BertTokenizer.from_pretrained(bio_bert_model)
model_bio= BertModel.from_pretrained(bio_bert_model)


In [5]:
med_bert_model='microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext'

tokenizer_med= BertTokenizer.from_pretrained(med_bert_model)
model_med= BertModel.from_pretrained(med_bert_model)


## Pre Processing

### On utilise les fonctions tokenize propre à chaque modèle BERT.

return_tensors='pt' : Ce paramètre indique au tokenizer de retourner des tenseurs PyTorch. Les tokens résultants seront encapsulés dans des tenseurs PyTorch.

padding=True : Ce paramètre garantit que les séquences tokenizées sont remplies de zéros jusqu'à la longueur maximale dans le batch. 

truncation=True : Si la séquence d'entrée est plus longue que la max_length spécifiée, elle sera tronquée pour s'ajuster. En définissant truncation=True, le tokenizer tronquera les séquences qui dépassent la max_length spécifiée.

max_length=512 : Ce paramètre fixe la longueur maximale des séquences. Si une séquence est plus longue que cela, elle sera tronquée ou découpée, selon le comportement du tokenizer.

### Embedding: Puis on récupère un tensor correspondant à la moyenne du pooling à travers les tokens du dernier état caché du BERT model.

outputs.last_hidden_state : Il s'agit de la dernière couche d'états cachés produites par le modèle BERT. Il s'agit d'un tenseur de la forme [batch_size, sequence_length, hidden_size], où hidden_size est la taille des états cachés (par exemple, 768 pour les modèles de base BERT).

.mean(dim=1) : Cette opération calcule la moyenne le long de la deuxième dimension du tenseur, qui est la longueur de la séquence. Après cette opération, on obtient un tenseur de la forme [batch_size, hidden_size].


In [6]:
def get_bert_embeddings(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True,max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pool the token embeddings
    return embeddings


In [7]:

def run_bert_for_document_ranking(start_doc, end_doc, mod,tok):
    dic_doc, dic_req, dic_req_doc = loadNFCorpus()
    model=mod
    tokenizer=tok
    model.eval()
    docs_to_keep = []
    reqs_to_keep = []
    dic_req_doc_to_keep = defaultdict(dict)

    ndcg_top = 5
    i = start_doc

    for req_id in dic_req_doc:
        if i > (end_doc - start_doc):
            break
        for doc_id in dic_req_doc[req_id]:
            dic_req_doc_to_keep[req_id][doc_id] = dic_req_doc[req_id][doc_id]
            docs_to_keep.append(doc_id)
            i += 1
        reqs_to_keep.append(req_id)

    docs_to_keep = list(set(docs_to_keep))

    corpus_doc_vectors = []
    corpus_req_vectors = {}
    corpus_doc_name = []
    corpus_dico_doc_name = {}
    i = 0

    for k in docs_to_keep:
        doc_text = dic_doc[k]
        doc_embeddings = get_bert_embeddings(doc_text, model, tokenizer)
        corpus_doc_vectors.append(doc_embeddings)
        corpus_doc_name.append(k)
        corpus_dico_doc_name[k] = i
        i += 1

    corpus_req_name = []
    corpus_dico_req_name = {}
    i = 0

    for k in reqs_to_keep:
        req_text = dic_req[k]
        req_embeddings = get_bert_embeddings(req_text, model, tokenizer)
        corpus_req_vectors[k] = req_embeddings
        corpus_req_name.append(k)
        corpus_dico_req_name[k] = i
        i += 1

    corpus_doc_vectors = torch.cat(corpus_doc_vectors, dim=0)

    ndcg_bert_cumul = 0
    nb_req = 0

    for req in corpus_req_vectors:
        true_docs = np.zeros(len(corpus_doc_vectors))

        req_vector = corpus_req_vectors[req]
        if isinstance(req_vector, torch.Tensor):
            if req_vector.size() == (1, 768):  # Ensure req_vector has expected dimensions
                

                # Rank documents based on scores
                ranked_docs = [(doc_id, score) for doc_id, score in zip(docs_to_keep, doc_scores)]
                ranked_docs.sort(key=lambda x: x[1], reverse=True)  # Sort in descending order based on scores

                # Compute NDCG using the ranked documents
                for doc_id, score in ranked_docs[:ndcg_top]:
                    if req in dic_req_doc_to_keep and doc_id in dic_req_doc_to_keep[req]:
                        pos_doc_id = corpus_dico_doc_name[doc_id]
                        true_docs[pos_doc_id] = dic_req_doc_to_keep[req][doc_id]

                ndcg_bert_cumul += ndcg_score([true_docs], [doc_scores], k=ndcg_top)
                nb_req += 1

    ndcg_bert_cumul /= nb_req
    print("NDCG for document ranking with BERT  =", ndcg_bert_cumul)
    return ndcg_bert_cumul




## 1) BERT base

In [8]:
run_bert_for_document_ranking(0, nb_docs,model_base,tokenizer_base)

NDCG for document ranking with BERT  = 0.7852518407813259


0.7852518407813259

## 2) BioBERT for sentences similarities

In [9]:
run_bert_for_document_ranking(0, nb_docs,model_bio,tokenizer_bio)

NDCG for document ranking with BERT  = 0.7932967492743007


0.7932967492743007

## 3)  BioMedBERT

In [24]:
run_bert_for_document_ranking(0, 150,model_med,tokenizer_med)

NDCG for document ranking with BERT  = 0.9533814120500341


0.9533814120500341